In [1]:
import tensorflow as tf
print(tf.__version__)

import random
import numpy as np
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint 
from sklearn import metrics  
import time

import keras

import matplotlib.pyplot as plt
import helper

from keras.models import Model,Sequential
from keras.layers import Input
from keras.layers.core import Dropout,Flatten,Dense
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.metrics import confusion_matrix,classification_report
from keras.applications.densenet import DenseNet201
from keras.applications.resnet import ResNet152

from keras.preprocessing.image import ImageDataGenerator

2.6.0


ModuleNotFoundError: No module named 'grp'

In [2]:
import os
data_dir = "D:\\Minor Project\\archive\\Train\\New Plant Diseases Dataset(Augmented)"
train_dir = data_dir + "/train"
valid_dir = data_dir + "/valid"
diseases = os.listdir(train_dir)
diseases

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spid

In [3]:
print("Total classification classes : {}".format(len(diseases)))

Total classification classes : 38


In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen_aug = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   horizontal_flip = True)

test_datagen_aug = ImageDataGenerator( rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range = 20,
                                   horizontal_flip = True)

In [6]:
training_set_aug = train_datagen_aug.flow_from_directory(directory= train_dir,
                                               target_size=(224, 224), 
                                               batch_size=50,
                                               class_mode='categorical') 
label_map = (training_set_aug.class_indices)
print("Target Classes Mapping Dict:\n")
print(label_map)

test_set_aug = test_datagen_aug.flow_from_directory(directory= valid_dir,
                                               target_size=(224, 224), 
                                               batch_size=50,
                                               class_mode='categorical',
                                               shuffle=False)

Found 70295 images belonging to 38 classes.
Target Classes Mapping Dict:

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15, 'Peach___Bacterial_spot': 16, 'Peach___healthy': 17, 'Pepper,_bell___Bacterial_spot': 18, 'Pepper,_bell___healthy': 19, 'Potato___Early_blight': 20, 'Potato___Late_blight': 21, 'Potato___healthy': 22, 'Raspberry___healthy': 23, 'Soybean___healthy': 24, 'Squash___Powdery_mildew': 25, 'Strawberry___Leaf_scorch': 26, 'Strawberry___healthy': 27, 'Tomato___Bacterial_s

In [7]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0025, patience=2, verbose=1)

In [8]:
from keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Conv2D, Input, Dense, BatchNormalization, MaxPool2D, GlobalAveragePooling2D,Dropout
from tensorflow.keras import Model

base_model = ResNet50(
                        include_top=False, 
                        weights='imagenet',
                        input_shape=(224, 224, 3), 
                        pooling="avg"
)

base_model.trainable = False
image_input = Input(shape=(224, 224, 3))

x = base_model(image_input,training = False)

x = Dense(256,activation = "relu")(x)
x = Dropout(0.2)(x)

x = Dense(128,activation = "relu")(x)
x = Dropout(0.2)(x)

image_output = Dense(38,activation="softmax")(x)

model = Model(image_input,image_output)

94781440/94765736 [==============================] - 76s 1us/step


In [9]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                4902  

In [ ]:
hist = model.fit(training_set_aug, epochs=20, validation_data=test_set_aug, callbacks=[callback])

Epoch 1/20
1406/1406 [==============================] - 10247s 7s/step - loss: 3.3602 - accuracy: 0.0734 - val_loss: 3.1442 - val_accuracy: 0.1217
Epoch 2/20
 274/1406 [====>.........................] - ETA: 6:11:35 - loss: 3.1023 - accuracy: 0.1247